In [1]:
import matplotlib.pyplot as plt #Visulization
from matplotlib import rc # 한글 깨져서
import seaborn as sns #Visulization
import pandas as pd
import numpy as np

import scipy as sp
import scipy.stats

import datetime as dt
from dateutil.relativedelta import relativedelta

import tensorflow as tf

tf.set_random_seed(2019)
rc('font', family="Malgun Gothic")

In [2]:
# mean_of_month = pd.read_csv("C:/Users/PC/Desktop/d/mean_of_month_1.csv")
# crwaling = pd.read_csv("C:/Users/PC/Desktop/d/농민신문 크롤링.csv")
mean_of_month = pd.read_csv("C:/Users/student/Desktop/d/mean_of_month_1.csv")
crwaling = pd.read_csv("C:/Users/student/Desktop/d/농민신문 크롤링.csv")

In [3]:
yyyymm_1 = '200801'
yyyymm_2 = '201812'

MONTH = relativedelta(months=+1)

fmt = '%Y%m'
date_1 = dt.datetime.strptime(yyyymm_1, fmt).date()
date_2 = dt.datetime.strptime(yyyymm_2, fmt).date()

yyyymm = []
while date_1 <= date_2:
    yyyymm.append(date_1.strftime('%Y')+date_1.strftime('%m'))
    date_1 += MONTH

In [4]:
mean_of_month['date'] = yyyymm
mean_of_month['yyyy'] = mean_of_month['date'].map(lambda x : x[0:4])
mean_of_month['mm'] = mean_of_month['date'].map(lambda x : x[4:6])

In [5]:
x_t = mean_of_month['kgPrice'][3:]
x_t1 = mean_of_month['kgPrice'][2:]
x_t2 = mean_of_month['kgPrice'][1:-1]
x_t3 = mean_of_month['kgPrice'][:-2]

x_train = np.c_[ x_t1.values[:-26], x_t2.values[:-26], x_t3.values[:-26] ]
y_train = x_t.values[:-26].reshape(-1,1)
x_test = np.c_[ x_t1.values[103:], x_t2.values[103:], x_t3.values[103:] ]
y_test = x_t.values[103:].reshape(-1,1)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(104, 3)
(103, 1)
(27, 3)
(26, 1)


In [6]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_normal([x_train.shape[1], 100]))
b1 = tf.Variable(tf.zeros([100]))
H1 = tf.nn.relu(tf.matmul(X, W1)+b1)

W2 = tf.Variable(tf.random_normal([100, 50]))
b2 = tf.Variable(tf.zeros([50]))
H2 = tf.nn.relu(tf.matmul(H1, W2)+b2)

W3 = tf.Variable(tf.random_normal([50, 100]))
b3 = tf.Variable(tf.zeros([50]))
H3 = tf.nn.relu(tf.matmul(H2, W3)+b3)

W4 = tf.Variable(tf.random_normal([100, 1]))
b4 = tf.Variable(tf.zeros([1]))
model = tf.matmul(H3, W4) + b4

Instructions for updating:
Colocations handled automatically by placer.


ValueError: Dimensions must be equal, but are 100 and 50 for 'add_2' (op: 'Add') with input shapes: [?,100], [50].

In [ ]:
cost = tf.reduce_mean(tf.square(model - Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
train_op = optimizer.minimize(cost)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for step in range(100000):
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_train, Y: y_train})
        if step % 10000 == 0: print("[step: {}	loss: {}]".format(step, cost_val))
    # 학습결과 확인
    predict = sess.run(model, feed_dict={X: x_test})
#     print("Y:", sess.run(model, feed_dict={X:x_test }))

In [ ]:
compare_value = np.c_[y_test, predict]

# numppy array에 컬럼이름 주기
compare_value = {'실제값':compare_value[:,0], '예측값':compare_value[:,1]}

# Dataframe으로 만들기
compare_value = pd.DataFrame(compare_value)

In [ ]:
compare_value['실제값_비율'] = -1
compare_value['예측값_비율'] = -1
compare_value['맞았니'] = ''

In [ ]:
for i in range(len(compare_value) - 1):
    if(compare_value.iloc[i,0] - compare_value.iloc[i+1,0] > 0):
        compare_value.iloc[i,2] = 0
    else: compare_value.iloc[i,2] = 1
    if(compare_value.iloc[i,1] - compare_value.iloc[i+1,1] > 0):
        compare_value.iloc[i,3] = 0
    else: compare_value.iloc[i,3] = 1

In [ ]:
for i in range(len(compare_value)):
    if(compare_value.iloc[i,2] == compare_value.iloc[i,3]):
        compare_value.iloc[i,4] = 1
    else:
        compare_value.iloc[i,4] = 0

In [ ]:
compare_value.iloc[:-1,4].sum() / (x_test.shape[0] - 1)

In [ ]:
plt.figure(num=None, figsize=(10, 6))
sns.lineplot(data=compare_value.iloc[:,[0,1]])

## 한달 땡겨봄

In [ ]:
compare_value2 = pd.DataFrame(compare_value.iloc[1:,0].reset_index(drop=True))

In [ ]:
compare_value2 = pd.concat((compare_value2,compare_value.iloc[:,1]), axis=1)

In [ ]:
plt.figure(num=None, figsize=(10, 6))
sns.lineplot(data=compare_value2)